# JSON - voor de microbit

Syntax voor eenvoudige JSON:

```
value : dictionary ; array ; simple .
simple: string ; number ; boolean .
string: '"', non-quote-seq, '"'.
int: digit-seq .
bool: 'false' ; 'true' .
dict: '{', keyvalue list option, '}' .
keyvalue: string, ':', value.
array: '[', value list option, ']' .
```

Extra voorwaarden:

* de values van een array moeten van hetzelfde type zijn (?)
* twee dict's zijn (per definitie) van hetzelfde type (?)
* whitespace heeft alleen betekenis in een string.

Hoe worden niet-aanwezige elementen in een array aangeduid in JSON? en in Python?

> Dit is evt. van belang als we voor channels arrays willen gebruiken in plaats van dictionairies.

Hieronde het bestand `json.py` zoals gebruikt in de gateway.

In [67]:
def loads(data: str):
    
    eof = chr(0)
    ch = data[0]
    pos = 0

    def error():
        raise Exception

    def nextch():
        nonlocal pos, ch
        pos = pos + 1
        if pos < len(data):
            ch = data[pos]
        else:
            ch = eof
        return

    def skipblanks():
        while ch == ' ' or ch == '\n' or ch == '\r' or ch == '\t':
            nextch()
    
    def accept(c: str):
        if ch != c:
            error()
        nextch()    
        skipblanks()
            
    def value():
        skipblanks()
        if ch == '{':
            return dictionary({})
        elif ch == '[':
            return array([])
        elif ch == '"':
            return string()
        elif '0' <= ch and ch <= '9':
            return number()
        elif ch == 'f' or ch == 't':
            return boolean()
        else:
            error()
        
    def keyvalue(items: dict):
        key = string()
        accept(':')
        val = value()
        items[key] = val
        return items

    def dictionary(items: dict):
        accept('{')
        if ch != '}':
            items = keyvalue(items)
            while ch == ',':
                accept(',')
                items = keyvalue(items)
        accept('}')
        return items
    
    def array(items: list):
        accept('[')
        if ch != ']':
            items.append(value())
            while ch == ',':
                accept(',')
                items.append(value())
        accept(']')
        return items

    def string():
        accept('"')
        value = ''
        while ch != '"':
            value += ch
            nextch()
        accept('"')
        return value

    def number():
        value = ''
        while '0' <= ch and ch <= '9':
            value += ch
            nextch()
        skipblanks()
        return int(value)

    def boolean():
        value = ''
        while 'a' <= ch and ch <= 'z':
            value += ch
            nextch()
        skipblanks()    
        if value == 'true':
            return True
        elif value == 'false':
            return False
        else:
            error()
    
    return value()



def dumps(val: dict) -> str:
    
    def to_json_keys (obj: dict) -> str:
        new_obj = {}
        for key in obj:
            value = obj[key]
            if type(value) is dict:
                value = to_json_keys(value)
            if type(key) is int:
                new_obj[str(key)] = value
            else:
                new_obj[key] = value
        return new_obj
            
    return str(to_json_keys(val)).replace("'", '"')       

Omzetten van een Python-dictionary naar "json--":

* vervangen van de integer keys door string-keys
* omzetten naar een string, op de standaard-Python manier.
* NB: we moeten wel de enkele quotes vervangen door dubbele...
* ik weet niet of we enige invloed hebben op het gebruik van spaties, of op de layout.

## Testen

In [68]:
test1 = '{ "aap": 10, "noot": {"mies": false, "teun": [1,2,3] }}'

In [69]:
loads(test1)

{'aap': 10, 'noot': {'mies': False, 'teun': [1, 2, 3]}}

In [70]:
test2 = '{ "aap": 10, "noot": {"mies": false, "teun": [ {  } ] }}'
loads(test2)

{'aap': 10, 'noot': {'mies': False, 'teun': [{}]}}

NB: we moeten nog een invariant bedenken m.b.t. "skipblanks": altijd aan het eind van een functie?

In [71]:
loads('{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}')

{'nodeid': 'fe3d',
 'counter': 3027,
 'payload': {'0': {'temperature': 235},
  '1': {'barometer': 10093},
  '2': {'dOut': 1},
  '8': {'aOut': 255}}}

In [76]:
str1 = dumps(loads('{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}'))
str1

'{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}'

Vergelijking met de officiële json-module:

In [73]:
import json

In [74]:
json.loads('{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}')

{'nodeid': 'fe3d',
 'counter': 3027,
 'payload': {'0': {'temperature': 235},
  '1': {'barometer': 10093},
  '2': {'dOut': 1},
  '8': {'aOut': 255}}}

Dit lijkt dus te werken (in elk geval voor goed-gevormde objecten).

In [77]:
str2 = json.dumps(json.loads('{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}'))
str2

'{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}'

In [78]:
str1 == str2

True

---

(onderstaande kan waarschijnlijk opgeruimd worden...)

## json--

Omzetten van een Python-dictionary naar "json--":

* vervangen van de integer keys door string-keys
* omzetten naar een string, op de standaard-Python manier.
* NB: we moeten wel de enkele quotes vervangen door dubbele...
* ik weet niet of we enige invloed hebben op het gebruik van spaties, of op de layout.

In [17]:
def to_json(obj: dict):
    new_obj = {}
    for key in obj:
        value = obj[key]
        if type(value) is dict:
            value = to_json(value)
        if type(key) is int:
            new_obj[str(key)] = value
        else:
            new_obj[key] = value
            
    return new_obj        

In [18]:
print(type(3))

<class 'int'>


In [19]:
my_obj = {"aa": 'bb', 'payload': {0: {'temp': 10}}}
str(to_json(my_obj)).replace("'", '"')

'{"aa": "bb", "payload": {"0": {"temp": 10}}}'

In [22]:
my_obj1 = {"nodeid": "fe3d",
 "counter": 3027,
 "payload": {
   0: {"temperature": 235},
   1: {"barometer": 10093},
   2: {"dOut": 1},
   8: {"aOut": 255}
  }
}

str(to_json(my_obj1)).replace("'", '"')

'{"nodeid": "fe3d", "counter": 3027, "payload": {"0": {"temperature": 235}, "1": {"barometer": 10093}, "2": {"dOut": 1}, "8": {"aOut": 255}}}'

In [34]:
print('".,."'.replace('"', '\\\"').replace(',','\\,'))

\".\,.\"


In [31]:
len('AT+MQTTPUB=0,"node/fe30/sensors","{\"nodeid\": \"fe30\", \"counter\": 82, \"payload\": \"{\"2\": \"{\"dIn\": 0}\", \"4\": \"{\"aIn\": 0}\", \"3\": \"{\"dIn\": 1}\", \"6\": \"{\"temperature\": 210}\"}\"}",0,0')

173

 {"nodeid": "fe30", "counter": 203, "payload": "{"2": "{"dIn": 0}", "4": "{"aIn": 0}", "3": "{"dIn": 0}", "6": "{"temperature": 200}"}"}